In [29]:
import sklearn
import numpy as np
import pandas as pd
import os
import requests
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.neural_network import MLPClassifier
from sklearn import svm

模型想法：


In [3]:
ls

README.md                          extractVolumesForSubtypes.ipynb
aimedSubFolderNameListMake.sh      makeSpecificPeopleCSV.ipynb
atrophyModelBuild.ipynb            merge_folder.ipynb
convertFreesurferOutput2CSV.ipynb  nohup.out
csvFiles/                          uploadNiiToBrainIo.ipynb
dicomToNii.ipynb                   uploadNiiToBrainIo.sh
downloadFreesurferOutput.sh


In [4]:
ls ./csvFiles/

ADNIMERGE.csv
AD_all.csv
AM1.csv
CM1.csv
CN_amyloid_pos.csv
Freesurfer_S1200_Extensively_Processed_fMRI_Data.csv
MCISubtypesFeatures.csv
MCI_all.csv
MCI_amyloid_pos.csv
MCI_amyloid_pos_freeStatsOut.csv
MM1.csv
Stats_S1200_Extensively_Processed_fMRI_Data.csv
freeStatsOut.csv


In [5]:
csvPath = "/home/ubuntu/Desktop/ADNI_MRI_Image/neuroScript/csvFiles"

In [6]:
volner = pd.read_csv(os.path.join(csvPath,"Freesurfer_S1200_Extensively_Processed_fMRI_Data.csv"))
MCI_freesurfer_Out = pd.read_csv(os.path.join(csvPath,"MCI_amyloid_pos_freeStatsOut.csv"))

In [66]:
all_data = pd.concat([volner, MCI_freesurfer_Out], axis=0, sort=False)

In [68]:
all_data.to_csv(os.path.join(csvPath,'all_data_0307.csv'))

In [65]:
MCI_freesurfer_Out

,Unnamed: 0,subject,FS_L_ThalamusProper_Vol,FS_L_Caudate_Vol,FS_L_Putamen_Vol,FS_L_Pallidum_Vol,FS_L_Hippo_Vol,FS_L_Amygdala_Vol,FS_L_AccumbensArea_Vol,FS_R_ThalamusProper_Vol,...,FS_R_Rostralmiddlefrontal_Area,FS_R_Superiorfrontal_Area,FS_R_Superiorparietal_Area,FS_R_Superiortemporal_Area,FS_R_Supramarginal_Area,FS_R_Frontalpole_Area,FS_R_Temporalpole_Area,FS_R_Transversetemporal_Area,FS_R_Insula_Area,FS_BrainSeg_Vol_No_Vent_Surf
0,0,021_S_4245,6296.8,3427.1,4168.1,2556.6,3525.7,1248.3,486.6,5873.3,...,4574.0,5905.0,4876.0,3203.0,3472.0,329.0,353.0,292.0,2544.0,9.565596e+05
1,1,021_S_4419,6355.7,3939.0,4269.3,1856.0,3874.7,1638.4,525.0,5700.6,...,4941.0,5787.0,4231.0,3177.0,2920.0,274.0,449.0,320.0,2258.0,9.452903e+05
2,2,013_S_2389,6008.1,2515.6,4192.6,1795.7,3580.6,1403.5,391.3,5946.3,...,5190.0,6211.0,4964.0,3446.0,3044.0,244.0,354.0,292.0,2103.0,9.177545e+05
3,3,003_S_6606,5703.9,2811.5,3783.9,1467.0,2896.3,1041.4,313.1,5881.1,...,4695.0,5895.0,5028.0,2691.0,3120.0,241.0,379.0,249.0,1833.0,8.796575e+05
4,4,099_S_4157,4807.3,3346.7,4066.6,1746.0,2894.4,1060.9,295.0,4948.7,...,4334.0,6438.0,4134.0,3568.0,3422.0,253.0,429.0,327.0,2182.0,9.010101e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,297,141_S_4426,7368.6,3982.6,4524.6,1627.0,2982.5,1510.9,442.5,7589.6,...,6825.0,7002.0,4519.0,3813.0,3704.0,294.0,513.0,258.0,2952.0,1.121195e+06
298,298,023_S_6356,7686.8,4738.4,3812.3,1858.8,3815.3,1123.5,378.1,8796.4,...,5437.0,6903.0,5434.0,3859.0,4258.0,273.0,443.0,397.0,3015.0,1.073383e+06
299,299,023_S_4035,5958.5,4286.9,4334.2,1693.6,3498.1,1187.7,298.3,6067.6,...,6076.0,6806.0,4881.0,3914.0,4030.0,318.0,400.0,373.0,2876.0,9.811456e+05
300,300,011_S_2274,7987.8,4394.6,5140.9,2189.9,3345.7,1107.4,566.6,7631.4,...,9367.0,9525.0,6884.0,4609.0,4578.0,367.0,557.0,489.0,3066.0,1.415997e+06


In [7]:
volner

,Subject,FS_IntraCranial_Vol,FS_BrainSeg_Vol,FS_BrainSeg_Vol_No_Vent,FS_BrainSeg_Vol_No_Vent_Surf,FS_LCort_GM_Vol,FS_RCort_GM_Vol,FS_TotCort_GM_Vol,FS_SubCort_GM_Vol,FS_Total_GM_Vol,...,FS_R_Rostralanteriorcingulate_Area,FS_R_Rostralmiddlefrontal_Area,FS_R_Superiorfrontal_Area,FS_R_Superiorparietal_Area,FS_R_Superiortemporal_Area,FS_R_Supramarginal_Area,FS_R_Frontalpole_Area,FS_R_Temporalpole_Area,FS_R_Transversetemporal_Area,FS_R_Insula_Area
0,100004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100206,1.864518e+06,1406924.0,1389007.0,1.386836e+06,304488.0,305634.0,610122.0,70240.0,807245.0,...,791.0,8627.0,8515.0,6315.0,4337.0,3522.0,356.0,500.0,384.0,2267.0
2,100307,1.512540e+06,1109710.0,1098854.0,1.097122e+06,240030.0,243207.0,483238.0,56623.0,664124.0,...,505.0,5702.0,6463.0,4538.0,3441.0,3613.0,257.0,412.0,375.0,2385.0
3,100408,1.684117e+06,1280171.0,1260943.0,1.259250e+06,260695.0,265243.0,525938.0,63869.0,726206.0,...,631.0,7092.0,7484.0,5367.0,3984.0,4144.0,298.0,410.0,421.0,2578.0
4,100610,1.816044e+06,1321362.0,1306929.0,1.304569e+06,276506.0,278002.0,554508.0,74869.0,762308.0,...,629.0,7027.0,7088.0,6820.0,3876.0,3730.0,298.0,393.0,337.0,2792.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,992774,1.519017e+06,1136193.0,1115715.0,1.113862e+06,232242.0,235740.0,467982.0,57546.0,647936.0,...,777.0,5664.0,6563.0,4838.0,3367.0,3384.0,199.0,342.0,247.0,2241.0
1202,993675,1.408199e+06,1037401.0,1020944.0,1.019593e+06,233533.0,246215.0,479748.0,54442.0,634223.0,...,551.0,4961.0,6206.0,6121.0,3113.0,3187.0,258.0,449.0,288.0,2091.0
1203,994273,1.738013e+06,1234076.0,1217881.0,1.216379e+06,258246.0,268631.0,526877.0,63592.0,722589.0,...,817.0,5369.0,7111.0,5860.0,3389.0,3858.0,220.0,435.0,386.0,2821.0
1204,995174,1.745369e+06,1253571.0,1240861.0,1.239238e+06,264143.0,270207.0,534350.0,66859.0,727971.0,...,570.0,6393.0,6837.0,4741.0,3565.0,3819.0,329.0,407.0,273.0,2550.0


In [11]:
features = ['FS_L_ThalamusProper_Vol', 'FS_L_Caudate_Vol', 'FS_L_Putamen_Vol', 'FS_L_Pallidum_Vol', 
            'FS_L_Hippo_Vol', 'FS_L_Amygdala_Vol', 'FS_L_AccumbensArea_Vol', 'FS_R_ThalamusProper_Vol', 
            'FS_R_Caudate_Vol', 'FS_R_Putamen_Vol', 'FS_R_Pallidum_Vol', 'FS_R_Hippo_Vol', 'FS_R_Amygdala_Vol', 
            'FS_R_AccumbensArea_Vol', 'FS_L_Bankssts_Thck', 'FS_L_Caudalanteriorcingulate_Thck', 
            'FS_L_Caudalmiddlefrontal_Thck', 'FS_L_Cuneus_Thck', 'FS_L_Entorhinal_Thck', 'FS_L_Fusiform_Thck', 
            'FS_L_Inferiorparietal_Thck', 'FS_L_Inferiortemporal_Thck', 'FS_L_Isthmuscingulate_Thck', 
            'FS_L_Lateraloccipital_Thck', 'FS_L_Lateralorbitofrontal_Thck', 'FS_L_Lingual_Thck', 
            'FS_L_Medialorbitofrontal_Thck', 'FS_L_Middletemporal_Thck', 'FS_L_Parahippocampal_Thck', 
            'FS_L_Paracentral_Thck', 'FS_L_Parsopercularis_Thck', 'FS_L_Parsorbitalis_Thck', 
            'FS_L_Parstriangularis_Thck', 'FS_L_Pericalcarine_Thck', 'FS_L_Postcentral_Thck', 
            'FS_L_Posteriorcingulate_Thck', 'FS_L_Precentral_Thck', 'FS_L_Precuneus_Thck', 
            'FS_L_Rostralanteriorcingulate_Thck', 'FS_L_Rostralmiddlefrontal_Thck', 'FS_L_Superiorfrontal_Thck', 
            'FS_L_Superiorparietal_Thck', 'FS_L_Superiortemporal_Thck', 'FS_L_Supramarginal_Thck', 
            'FS_L_Frontalpole_Thck', 'FS_L_Temporalpole_Thck', 'FS_L_Transversetemporal_Thck', 
            'FS_L_Insula_Thck', 'FS_R_Bankssts_Thck', 'FS_R_Caudalanteriorcingulate_Thck', 
            'FS_R_Caudalmiddlefrontal_Thck', 'FS_R_Cuneus_Thck', 'FS_R_Entorhinal_Thck', 'FS_R_Fusiform_Thck', 
            'FS_R_Inferiorparietal_Thck', 'FS_R_Inferiortemporal_Thck', 'FS_R_Isthmuscingulate_Thck', 
            'FS_R_Lateraloccipital_Thck', 'FS_R_Lateralorbitofrontal_Thck', 'FS_R_Lingual_Thck', 
            'FS_R_Medialorbitofrontal_Thck', 'FS_R_Middletemporal_Thck', 'FS_R_Parahippocampal_Thck', 
            'FS_R_Paracentral_Thck', 'FS_R_Parsopercularis_Thck', 'FS_R_Parsorbitalis_Thck', 
            'FS_R_Parstriangularis_Thck', 'FS_R_Pericalcarine_Thck', 'FS_R_Postcentral_Thck', 
            'FS_R_Posteriorcingulate_Thck', 'FS_R_Precentral_Thck', 'FS_R_Precuneus_Thck', 
            'FS_R_Rostralanteriorcingulate_Thck', 'FS_R_Rostralmiddlefrontal_Thck', 
            'FS_R_Superiorfrontal_Thck', 'FS_R_Superiorparietal_Thck', 'FS_R_Superiortemporal_Thck',
            'FS_R_Supramarginal_Thck', 'FS_R_Frontalpole_Thck', 'FS_R_Temporalpole_Thck', 
            'FS_R_Transversetemporal_Thck', 'FS_R_Insula_Thck']

In [12]:
##Convert pd.Dataframe to feature(np.array) here
def Convert2FeatureVector(pdFrame):
    pdFrame = pdFrame.dropna()
    pdFrame['reverse_FS_BrainSeg_Vol_No_Vent_Surf'] = 1.0 / pdFrame['FS_BrainSeg_Vol_No_Vent_Surf']
    X = []
    x =[]
    weighted_mean_feature_dict = {}
    for feature in features:
        if feature.split('_')[-1] == 'Thck':
            feature_area = feature[0:-4] + 'Area'
            weighted_mean_feature_dict[feature] = (pdFrame[feature] * pdFrame[feature_area]).sum() / pdFrame[feature_area].sum()
        if feature.split('_')[-1] == 'Vol':
            weighted_mean_feature_dict[feature] = (pdFrame[feature] * pdFrame['reverse_FS_BrainSeg_Vol_No_Vent_Surf']).sum() / pdFrame['reverse_FS_BrainSeg_Vol_No_Vent_Surf'].sum()   
    variance_feature_dict = {}
    for feature in features:
        if feature.split('_')[-1] == 'Thck':
            feature_area = feature[0:-4] + 'Area'
            variance_feature_dict[feature] = np.sqrt((pdFrame[feature_area] * np.square(pdFrame[feature] - weighted_mean_feature_dict[feature])).sum() / (pdFrame[feature_area].sum() - (np.square(pdFrame[feature_area])).sum() / pdFrame[feature_area].sum()))
        if feature.split('_')[-1] == 'Vol':
            variance_feature_dict[feature] = np.sqrt((pdFrame["reverse_FS_BrainSeg_Vol_No_Vent_Surf"] * np.square(pdFrame[feature] - weighted_mean_feature_dict[feature])).sum() / (pdFrame["reverse_FS_BrainSeg_Vol_No_Vent_Surf"].sum() - (np.square(pdFrame["reverse_FS_BrainSeg_Vol_No_Vent_Surf"])).sum() / pdFrame["reverse_FS_BrainSeg_Vol_No_Vent_Surf"].sum()))
    for index,row in pdFrame.iterrows():
        for feature in features:
            x.append((np.float(row[feature]) - weighted_mean_feature_dict[feature]) / variance_feature_dict[feature])
        X.append(x)
        x =[]
        if len(X) % 100 == 0:
            print("finish 100")
    X = np.array(X)
    return X



In [18]:
MCI_vector = Convert2FeatureVector(MCI_freesurfer_Out)
MCI_vector.shape

finish 100
finish 100
finish 100


(302, 82)

In [17]:
volner_vector = Convert2FeatureVector(volner)
volner_vector.shape

/home/ubuntu/anaconda3/envs/spm/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100


(1113, 82)

In [61]:
MCI_label = np.ones(302)
volner_label = np.zeros(1113)
print(MCI_label.shape,volner_label.shape)
y = np.concatenate((MCI_label,volner_label))
print(y.shape)

X = np.concatenate((MCI_vector,volner_vector))
X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

(302,) (1113,)
(1415,)


In [62]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(64,128), random_state=1,max_iter=700)
clf.fit(X_train,y_train)
clf.score(X_test, y_test)

0.7435294117647059

In [58]:
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(64,100), random_state=1,max_iter=700).fit(X_train, y_train)
clf.score(X_test, y_test)


0.9777777777777777

In [8]:
volner_no_Nan = volner.dropna()

In [15]:
weighted_mean_feature_dict

{'FS_L_ThalamusProper_Vol': 8383.401946037651,
 'FS_L_Caudate_Vol': 3779.548552416655,
 'FS_L_Putamen_Vol': 5477.463380741111,
 'FS_L_Pallidum_Vol': 1352.1266204666742,
 'FS_L_Hippo_Vol': 4388.28864820358,
 'FS_L_Amygdala_Vol': 1540.2196256337984,
 'FS_L_AccumbensArea_Vol': 558.5932627442517,
 'FS_R_ThalamusProper_Vol': 7333.339447460327,
 'FS_R_Caudate_Vol': 3898.7400001920596,
 'FS_R_Putamen_Vol': 5538.65931233499,
 'FS_R_Pallidum_Vol': 1483.5419608719665,
 'FS_R_Hippo_Vol': 4453.4554336924875,
 'FS_R_Amygdala_Vol': 1620.485301293954,
 'FS_R_AccumbensArea_Vol': 593.2909656174246,
 'FS_L_Bankssts_Thck': 2.6787044633589017,
 'FS_L_Caudalanteriorcingulate_Thck': 2.688940644010922,
 'FS_L_Caudalmiddlefrontal_Thck': 2.731166869902166,
 'FS_L_Cuneus_Thck': 2.0782934679172,
 'FS_L_Entorhinal_Thck': 3.290457523055491,
 'FS_L_Fusiform_Thck': 2.878015910355798,
 'FS_L_Inferiorparietal_Thck': 2.5746061419971324,
 'FS_L_Inferiortemporal_Thck': 2.9367398454142357,
 'FS_L_Isthmuscingulate_Thck': 2

In [14]:
weighted_mean_feature_dict = {}
for feature in features:
    if feature.split('_')[-1] == 'Thck':
        feature_area = feature[0:-4] + 'Area'
        weighted_mean_feature_dict[feature] = (volner_no_Nan[feature] * volner_no_Nan[feature_area]).sum() / volner_no_Nan[feature_area].sum()
    if feature.split('_')[-1] == 'Vol':
        weighted_mean_feature_dict[feature] = (volner_no_Nan[feature] * volner_no_Nan['reverse_FS_BrainSeg_Vol_No_Vent_Surf']).sum() / volner_no_Nan['reverse_FS_BrainSeg_Vol_No_Vent_Surf'].sum()   
    

In [16]:
variance_feature_dict = {}
for feature in features:
    if feature.split('_')[-1] == 'Thck':
        feature_area = feature[0:-4] + 'Area'
        variance_feature_dict[feature] = np.sqrt((volner_no_Nan[feature_area] * np.square(volner_no_Nan[feature] - weighted_mean_feature_dict[feature])).sum() / (volner_no_Nan[feature_area].sum() - (np.square(volner_no_Nan[feature_area])).sum() / volner_no_Nan[feature_area].sum()))
    if feature.split('_')[-1] == 'Vol':
        variance_feature_dict[feature] = np.sqrt((volner_no_Nan["reverse_FS_BrainSeg_Vol_No_Vent_Surf"] * np.square(volner_no_Nan[feature] - weighted_mean_feature_dict[feature])).sum() / (volner_no_Nan["reverse_FS_BrainSeg_Vol_No_Vent_Surf"].sum() - (np.square(volner_no_Nan["reverse_FS_BrainSeg_Vol_No_Vent_Surf"])).sum() / volner_no_Nan["reverse_FS_BrainSeg_Vol_No_Vent_Surf"].sum()))


In [9]:
volner_no_Nan['reverse_FS_BrainSeg_Vol_No_Vent_Surf'] = 1.0 / volner_no_Nan['FS_BrainSeg_Vol_No_Vent_Surf']

/home/ubuntu/anaconda3/envs/spm/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
variance_feature_dict

{'FS_L_ThalamusProper_Vol': 912.7455088328561,
 'FS_L_Caudate_Vol': 471.83902443396664,
 'FS_L_Putamen_Vol': 729.6000588675457,
 'FS_L_Pallidum_Vol': 234.6540458342281,
 'FS_L_Hippo_Vol': 481.4536162459448,
 'FS_L_Amygdala_Vol': 201.99980007613954,
 'FS_L_AccumbensArea_Vol': 92.72667858508443,
 'FS_R_ThalamusProper_Vol': 774.0151005112599,
 'FS_R_Caudate_Vol': 482.36992390011756,
 'FS_R_Putamen_Vol': 635.4332533471884,
 'FS_R_Pallidum_Vol': 199.6384508796309,
 'FS_R_Hippo_Vol': 457.26355341243976,
 'FS_R_Amygdala_Vol': 215.27805408927637,
 'FS_R_AccumbensArea_Vol': 99.35267964229074,
 'FS_L_Bankssts_Thck': 0.13724280772138953,
 'FS_L_Caudalanteriorcingulate_Thck': 0.17202413576091918,
 'FS_L_Caudalmiddlefrontal_Thck': 0.1217081559775747,
 'FS_L_Cuneus_Thck': 0.11453535376731407,
 'FS_L_Entorhinal_Thck': 0.24252647120052112,
 'FS_L_Fusiform_Thck': 0.11542213653784122,
 'FS_L_Inferiorparietal_Thck': 0.10750365432075935,
 'FS_L_Inferiortemporal_Thck': 0.12580944628248333,
 'FS_L_Isthmusci

In [26]:
X = []
x =[]
for index,row in volner_no_Nan.iterrows():
    for feature in features:
        x.append((np.float(row[feature]) - weighted_mean_feature_dict[feature]) / variance_feature_dict[feature])
    X.append(x)
    x =[]
    if len(X) % 100 == 0:
        print("finish 100")
X = np.array(X)      
print(X.shape)

finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
finish 100
(1113, 82)


In [ ]:
selectFeatures2 = ["left-wthi-bankssts","left-wthi-caudalanteriorcingulate","left-wthi-caudalmiddlefrontal",
                  "left-wthi-cuneus","left-wthi-entorhinal","left-wthi-fusiform","left-wthi-inferiorparietal",
                  "left-wthi-inferiortemporal","left-wthi-isthmuscingulate","left-wthi-lateraloccipital",
                  "left-wthi-lateralorbitofrontal","left-wthi-lingual","left-wthi-medialorbitofrontal",
                  "left-wthi-middletemporal","left-wthi-parahippocampal","left-wthi-paracentral",
                  "left-wthi-parsopercularis","left-wthi-parsorbitalis","left-wthi-parstriangularis",
                  "left-wthi-pericalcarine","left-wthi-postcentral","left-wthi-posteriorcingulate",
                  "left-wthi-precentral","left-wthi-precuneus","left-wthi-rostralanteriorcingulate",
                  "left-wthi-rostralmiddlefrontal","left-wthi-superiorfrontal","left-wthi-superiorparietal",
                  "left-wthi-superiortemporal","left-wthi-supramarginal","left-wthi-frontalpole",
                  "left-wthi-temporalpole","left-wthi-transversetemporal","left-wthi-insula","right-wthi-bankssts",
                  "right-wthi-caudalanteriorcingulate","right-wthi-caudalmiddlefrontal","right-wthi-cuneus",
                  "right-wthi-entorhinal","right-wthi-fusiform","right-wthi-inferiorparietal",
                  "right-wthi-inferiortemporal","right-wthi-isthmuscingulate","right-wthi-lateraloccipital",
                  "right-wthi-lateralorbitofrontal","right-wthi-lingual","right-wthi-medialorbitofrontal",
                  "right-wthi-middletemporal","right-wthi-parahippocampal","right-wthi-paracentral",
                  "right-wthi-parsopercularis","right-wthi-parsorbitalis","right-wthi-parstriangularis",
                  "right-wthi-pericalcarine","right-wthi-postcentral","right-wthi-posteriorcingulate",
                  "right-wthi-precentral","right-wthi-precuneus","right-wthi-rostralanteriorcingulate",
                  "right-wthi-rostralmiddlefrontal","right-wthi-superiorfrontal","right-wthi-superiorparietal",
                  "right-wthi-superiortemporal","right-wthi-supramarginal","right-wthi-frontalpole",
                  "right-wthi-temporalpole","right-wthi-transversetemporal","right-wthi-insula",
                  "right-svol-Thalamus","right-svol-Caudate","right-svol-Putamen","right-svol-Pallidum",
                  "right-svol-Hippocampus","right-svol-Amygdala","right-svol-Accumbens","left-svol-Thalamus",
                  "left-svol-Caudate","left-svol-Putamen","left-svol-Pallidum","left-svol-Hippocampus",
                  "left-svol-Amygdala","left-svol-Accumbens"]

In [ ]:
#load the jwt token (run bl login to create this file)
jwt_file = open(os.environ['HOME']+'/.config/brainlife.io/.jwt', mode='r')
jwt = jwt_file.read()
#query datasets records
find = { 
    '_group_id': '69', #see project detail page 
    'service': 'brainlife/app-freesurfer', 
    #'service_branch': '0.0.5',
    'status': 'finished' }
params = { 
    'limit': 100, 
    'select': 'config._inputs.meta', #we just want product and meta
    'find': json.JSONEncoder().encode(find) }
res = requests.get('https://brainlife.io/api/amaretti/task', params=params, headers={'Authorization': 'Bearer '+jwt})
if res.status_code != 200:
    raise Exception("failed to download datasets list:"+res.status_code)
tasks = res.json()["tasks"]
for task in tasks:
    taskid=task["_id"]
    subject=task["config"]["_inputs"][0]["meta"]["subject"]
    print(taskid, subject)
    url = 'https://brainlife.io/api/amaretti/task/download/'+taskid+'/freesurfer/output/stats?at='+jwt
    res = requests.get(url, allow_redirects=True)
    open(subject+'.tar.gz', 'wb').write(res.content)

In [ ]:
res = requests.get('https://brainlife.io/api/amaretti/resource',headers={'Authorization': 'Bearer '+jwt})

In [ ]:
#This verison is for all volume feature
selectFeatures1 = ["left-vol-bankssts","left-vol-caudalanteriorcingulate","left-vol-caudalmiddlefrontal",
                  "left-vol-cuneus","left-vol-entorhinal","left-vol-fusiform","left-vol-inferiorparietal",
                  "left-vol-inferiortemporal","left-vol-isthmuscingulate","left-vol-lateraloccipital",
                  "left-vol-lateralorbitofrontal","left-vol-lingual","left-vol-medialorbitofrontal",
                  "left-vol-middletemporal","left-vol-parahippocampal","left-vol-paracentral",
                  "left-vol-parsopercularis","left-vol-parsorbitalis","left-vol-parstriangularis",
                  "left-vol-pericalcarine","left-vol-postcentral","left-vol-posteriorcingulate",
                  "left-vol-precentral","left-vol-precuneus","left-vol-rostralanteriorcingulate",
                  "left-vol-rostralmiddlefrontal","left-vol-superiorfrontal","left-vol-superiorparietal",
                  "left-vol-superiortemporal","left-vol-supramarginal","left-vol-frontalpole",
                  "left-vol-temporalpole","left-vol-transversetemporal","left-vol-insula","right-vol-bankssts",
                  "right-vol-caudalanteriorcingulate","right-vol-caudalmiddlefrontal","right-vol-cuneus",
                  "right-vol-entorhinal","right-vol-fusiform","right-vol-inferiorparietal",
                  "right-vol-inferiortemporal","right-vol-isthmuscingulate","right-vol-lateraloccipital",
                  "right-vol-lateralorbitofrontal","right-vol-lingual","right-vol-medialorbitofrontal",
                  "right-vol-middletemporal","right-vol-parahippocampal","right-vol-paracentral",
                  "right-vol-parsopercularis","right-vol-parsorbitalis","right-vol-parstriangularis",
                  "right-vol-pericalcarine","right-vol-postcentral","right-vol-posteriorcingulate",
                  "right-vol-precentral","right-vol-precuneus","right-vol-rostralanteriorcingulate",
                  "right-vol-rostralmiddlefrontal","right-vol-superiorfrontal","right-vol-superiorparietal",
                  "right-vol-superiortemporal","right-vol-supramarginal","right-vol-frontalpole",
                  "right-vol-temporalpole","right-vol-transversetemporal","right-vol-insula",
                  "right-svol-Thalamus","right-svol-Caudate","right-svol-Putamen","right-svol-Pallidum",
                  "right-svol-Hippocampus","right-svol-Amygdala","right-svol-Accumbens","left-svol-Thalamus",
                  "left-svol-Caudate","left-svol-Putamen","left-svol-Pallidum","left-svol-Hippocampus",
                  "left-svol-Amygdala","left-svol-Accumbens"]